In [1]:
%autosave 0
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))



## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
57,ABW,Aruba,2020-05-19,101,0,3,0,945.994,0.000,28.099,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
188,AFG,Afghanistan,2020-05-19,7072,408,173,4,181.667,10.481,4.444,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
247,AGO,Angola,2020-05-19,48,0,2,0,1.460,0.000,0.061,...,2.405,1.362,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-18,France,142738.0,0.0,28347.0,0.0,0.0,0.0
0,2020-05-18,China,84129.0,0.0,4658.0,0.0,0.0,0.0
0,2020-05-18,Italy,226372.0,0.0,32070.0,0.0,0.0,0.0
0,2020-05-18,Spain,231843.0,0.0,27746.0,0.0,0.0,0.0
0,2020-05-18,United States,1512627.0,0.0,90617.0,0.0,0.0,0.0
0,2020-05-18,World,4781688.0,0.0,319613.0,0.0,0.0,0.0
0,2020-05-18,United Kingdom,247469.0,0.0,35022.0,0.0,0.0,0.0
0,2020-05-18,Germany,175312.0,0.0,7962.0,0.0,0.0,0.0
0,2020-05-18,Iran,122031.0,0.0,7032.0,0.0,0.0,0.0
0,2020-05-18,Turkey,150934.0,0.0,4176.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [6]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-19,France,142738.0,142903.0,28347.0,28239.0,-165.0,108.0
0,2020-05-19,China,84129.0,84063.0,4658.0,4638.0,66.0,20.0
0,2020-05-19,Italy,226372.0,225886.0,32070.0,32007.0,486.0,63.0
0,2020-05-18,Spain,231843.0,231606.0,27746.0,27709.0,237.0,37.0
0,2020-05-19,United States,1512627.0,1508598.0,90617.0,90353.0,4029.0,264.0
0,2020-05-19,World,4781688.0,4766468.0,319613.0,318201.0,15220.0,1412.0
0,2020-05-19,United Kingdom,247469.0,246406.0,35022.0,34796.0,1063.0,226.0
0,2020-05-19,Germany,175312.0,175210.0,7962.0,8007.0,102.0,-45.0
0,2020-05-19,Iran,122031.0,122492.0,7032.0,7057.0,-461.0,-25.0
0,2020-05-19,Turkey,150934.0,150593.0,4176.0,4171.0,341.0,5.0


In [7]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")